In [2]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from pymongo import MongoClient
import logging,re,time
import requests
#import textblob
#import langid
import collections
import numpy as np

pd.set_option('display.max_rows', 100) 
filepath = '/Users/Suad/FB-data/'
client = MongoClient()
db = client.unicief


logging.basicConfig(level=logging.INFO,
                    filename='log.log', # log to this file
                    format='%(asctime)s %(message)s') # include timestamp

pagesCollection = db.syrpages
postsCollection = db.syrposts
commentsCollection = db.syrcomments
likesCollection=db.syrlikes


print "loaded Records "
print pagesCollection.count()
#Set All False
pagesCollection.update_many({},{'$set':{'relevant':False}})
print "Set all false"
print pagesCollection.count()

slicee = DataFrame(list(pagesCollection.find()))

#[['name','category','talking_about_count','engagement','likes','start_info_clean']]
#print slicee
main_cat = ['Society/Culture Website', 'Travel/Leisure' ,'Transport/Freight', 'Work Position', 'Community', 
            'Small Business',
            'Society/Culture Website', 'Business Person', 'Business Services','Cause', 'Community Organization', 
            'Community/Government' , 'Company' , 'Consulting/Business Services', 'Regional Website', 'Media/News/Publishing'
             ,'Local/Travel Website', 'Local Business']

#false_kw =[u'عشاق','دوري','كرة','عشق','عاشق','الله','أبطال','فتيات','اسلام','ابطال','بنات'];
false_kw =u'عشاق|دوري|كرة|عشق|عاشق|الله|أبطال|فتيات|اسلام|ابطال|بنات'
#Loop pages
for pid in pagesCollection.distinct("id"):
    pagesCollection.update_one({'id':pid , 'category': { '$in': main_cat } },{'$set':{'relevant':True}})

print "Total pages after set relevant categories"
relevant= pagesCollection.find({'relevant':True})
print relevant.count()

import re
regexp = re.compile(false_kw)
for page in relevant:    
    if (regexp.search(page['name']) is not None):
        pagesCollection.update_one({'id':page['id']} ,{'$set':{'relevant':False}})
        #log 
        #print "removed:" + page['name']
    if (page['likes']==0):
        pagesCollection.update_one({'id':page['id']} ,{'$set':{'relevant':False}})    
        #log 
        # print "removed:" + page['name']
#for pid in relevant.distinct("id"):

print "Total pages after remove false kw and zero engagment"
relevant= pagesCollection.find({'relevant':True})
print relevant.count()    



loaded Records 
1151
Set all false
1151
Total pages after set relevant categories
479
Total pages after remove false kw and zero engagment
352


In [28]:
import matplotlib.pyplot as plt
from networkx.algorithms import *
limit =0
result =[]
relevant= pagesCollection.find({'relevant':True},{"id":1, "_id":0})

#relevant.toArrays(function(u){ result.push(u.text) })
B = nx.Graph()
usersG = nx.Graph()
relevant_pages=[]
relevant_posts=[]
posts =[]
net={}
for x in relevant:
    relevant_pages.append(x["id"]) # find better way todo this

postsCollection = db.syrposts
rpostsCollection = db.syrposts.find({'page_id': {'$in': relevant_pages }},{"id":1, "_id":0})
for post in rpostsCollection:
    relevant_posts.append(post["id"].split('_')[1])
    
print "Relevant Posts: ", len(relevant_posts), "out of " , postsCollection.count()
#print relevant_posts
commentsCollection = db.syrcomments
print "Comments count ", commentsCollection.count()
for comment in commentsCollection.find().limit(10000):
    ids = comment['id'].split('_')
    key = post_id = ids[0]
    if(key not in  relevant_posts):
        continue
    #p= postsCollection.find({'id':key},{"page_id":1, "_id":0})
    #print list(p)
    #post_id =ids[1]
    user = comment['from']['id']
    if key not in net:
        net[key] = [user]
    else:
        #print "found"
        if user not in net[key]:
            net[key].append(user)

B.add_nodes_from(net.keys(),Bipartite=0)
        
for post in net:
        B.add_nodes_from(list(net[post]),Bipartite=1)
        for value in net[post]:
            B.add_edge(post,value)
            
users_nodes, posts_nodes = bipartite.sets(B)
print "posts:",posts_nodes
print "users:",users_nodes
#nx.draw(B)
#plt.show()

Relevant Posts:  34508 out of  124232
Comments count  317666
posts: set([u'1632965850293641', u'611246989016208', u'1626930497563843', u'610883809052526', u'1628356587421234', u'1651292225127670', u'1651529618437264', u'611304552343785', u'611022989038608', u'610924329048474', u'1633936173529942', u'610974489043458', u'1648945992028960', u'610739499066957', u'611336909007216', u'611025312371709', u'611023422371898', u'611013522372888', u'611377015669872', u'1626969357559957', u'610722569068650', u'1646327988957427', u'1638534206403472', u'611213329019574', u'610719609068946', u'610937585713815', u'1633917590198467', u'610679342406306', u'748312285297473', u'1487776418190185', u'1644939875762905', u'1650507001872859', u'10153029456390988', u'611010365706537', u'1630136527243240', u'1642451786011714', u'610970232377217', u'610613935746180', u'1627263697530523'])
users: set([u'731437070335904', u'1090211661003456', u'941496229222518', u'815562801876378', u'1135768299785899', u'16276681608